In [1]:
import numpy as np

import matplotlib.pyplot as plt

from LIMxCMBL.cross_spectrum import *
from LIMxCMBL.init import *
from LIMxCMBL.noise import *
from LIMxCMBL.kernels import *

from scipy.signal.windows import dpss

from tqdm import tqdm

# CCAT-prime
zmin = 3.5
zmax = 8.1
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))
window = np.where((chis_resample > chimin) & (chis_resample < chimax))[0]

_chimin, _chimax = chis_resample[window][0], chis_resample[window][-1]


Lambdas = np.logspace(-5, 0, 50)
Lambda = Lambdas[39]
print(Lambda)



chis_resample_len = int(np.log2(len(chis_resample)))
print(chis_resample_len)
oup_fname = '/scratch/users/delon/LIMxCMBL/eHIeHI/zmin_%.5f_zmax_%.5f_Lambda_%.5e_chi_sample_2e%d'%(zmin, zmax, Lambda,chis_resample_len)
eIeI = np.load(oup_fname + 'eIeI.npy')
eIeLO = np.load(oup_fname + 'eIeLO.npy' ,)
eLOeI = np.load(oup_fname + 'eLOeI.npy' , )
eLOeLO = np.load(oup_fname + 'eLOeLO.npy', )
eHIeHI = eIeI + eLOeLO - eIeLO - eLOeI

IHi_kappa_fname = '/scratch/users/delon/LIMxCMBL/IHiKappa/zmin_%.5f_zmax_%.5f_Lambda_%.5f_chi_sample_2e%d.npy'%(zmin, zmax, Lambda,chis_resample_len)
IHi_kappa = np.load(IHi_kappa_fname) #[kJy/sr Mpc]
IHi_kappa_windowed = IHi_kappa[:, window].astype(np.float64)

0.09540954763499944
13


In [2]:
eHIeHI.shape

(1652, 1652)

In [3]:
chis_restricted = chis_resample[np.where((chis_resample >= chimin) & (chis_resample <= chimax))]

In [4]:
D = np.diag(chis_restricted)
d = D @ IHi_kappa_windowed.T
L = np.linalg.cholesky(D @ eHIeHI @ D)
y = np.linalg.solve(L, d)
x = np.linalg.solve(L.T, y)
res1 = np.einsum('ij, ji->i', d.T, x)

LinAlgError: Matrix is not positive definite

In [5]:
from scipy.interpolate import interp1d

f_d = interp1d(x = chis_restricted, y = d, axis = 0)

In [6]:
from scipy.interpolate import interp2d

In [7]:
f_eIeI = lambda chi, chip, dchi : 1 / (dchi * chi ** 2)
f_eIeLO = lambda chi, chip : 1/chi**2  * Lambda / np.pi * np.sinc(Lambda * (chi - chip) / np.pi)
f_eLOeI = lambda chi, chip : 1/chip**2 * Lambda / np.pi * np.sinc(Lambda * (chi - chip) / np.pi)

In [9]:
def comps(chis):
    _chis  = chis.reshape(-1, 1)
    _chips = chis.reshape(1 ,-1)
    dchi = np.mean(np.diff(chis))
    
    eIeI = np.diag(f_eIeI(chis, chis, dchi))
    eIeLO = f_eIeLO(_chis, _chips)
    eLOeI = f_eLOeI(_chis, _chips)
    
    return eIeI, eIeLO, eLOeI

a1, a2, a3 = comps(chis_restricted)

_chi = chis_restricted.reshape(-1, 1)
_chip = chis_restricted.reshape(1, -1)
def integrand(_chib): 
    return Lambda**2 / np.pi**2 / _chib ** 2 * np.sinc(Lambda * (_chi - _chib) / np.pi) * np.sinc(Lambda * (_chip - _chib) / np.pi)

a4, _ = quad_vec(integrand, chimin, chimax, epsabs = 0.0, epsrel = 1e-3, workers = 32)

eHIeHI_orig = a1 - a2 - a3 + a4
np.allclose(eHIeHI_orig, eHIeHI, atol = 0.0, rtol = 1e-2)

True

In [15]:
np.allclose(eHIeHI_orig, eHIeHI, atol = 0.0, rtol = 1e-6)

True

In [16]:
chis_dense = np.linspace(np.min(chis_restricted),
                        np.max(chis_restricted),
                        8 * len(chis_restricted))
a1, a2, a3 = comps(chis_dense)


_chi = chis_dense.reshape(-1, 1)
_chip = chis_dense.reshape(1, -1)
def integrand(_chib): 
    return Lambda**2 / np.pi**2 / _chib ** 2 * np.sinc(Lambda * (_chi - _chib) / np.pi) * np.sinc(Lambda * (_chip - _chib) / np.pi)

a4, _ = quad_vec(integrand, chimin, chimax, epsabs = 0.0, epsrel = 1e-3, workers = 32)

eHIeHI_dense = a1 - a2 - a3 + a4

KeyboardInterrupt: 

In [ ]:
D = np.diag(chis_dense)
d_dense = f_d(chis_dense)
L = np.linalg.cholesky(D @ eHIeHI_dense @ D)
y = np.linalg.solve(L, d_dense)
x = np.linalg.solve(L.T, y)
res2 = np.einsum('ij, ji->i', d_dense.T, x)

In [ ]:
plt.figure(dpi = 300)
# plt.plot(ells, ells**2 * res1, 'k')
plt.plot(ells, ells**2 * res2, 'r--')

plt.legend(frameon=False)
plt.xscale('log')